In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report

In [2]:
# from azureml import Workspace
# ws = Workspace(
#     workspace_id='1a6d99363f7348dc9687db8778179f8b',
#     authorization_token='5STQlt8JP6YC5x6USAs89ER4G5d8+iPCouHANhOaZpezlts8SmzjJ1SeTdDo4HtaMHfgluZ9Xp63itGckqyMog==',
#     endpoint='https://studioapi.azureml.net'
# )
# ds = ws.datasets['Hz-CTU13_6.csv']
# data = ds.to_dataframe()

# from azureml import Workspace
# ws = Workspace(
#     workspace_id='1a6d99363f7348dc9687db8778179f8b',
#     authorization_token='5STQlt8JP6YC5x6USAs89ER4G5d8+iPCouHANhOaZpezlts8SmzjJ1SeTdDo4HtaMHfgluZ9Xp63itGckqyMog==',
#     endpoint='https://studioapi.azureml.net'
# )
# ds = ws.datasets['Hz-CTU13_9.csv']
# data = ds.to_dataframe()

In [3]:
data = pd.read_csv('Hz-CTU13_6.csv')

In [4]:
print(data.shape)

(1492, 204)


In [5]:
data['original_label'] = data['original_label'].str.split("-").str[1]
data.loc[data['original_label'] != 'Botnet', 'original_label'] = 0
data.loc[data['original_label'] == 'Botnet', 'original_label'] = 1
data['original_label'] = data['original_label'].astype(int)

In [6]:
data['original_label'].unique()

array([1, 0], dtype=int64)

In [7]:
data.drop(['original_dataset', 'event_generator', 'event_signature', 'event_priority'], axis=1, inplace=True)

In [8]:
res = data['original_label']
data.drop('original_label', inplace=True, axis=1)

In [9]:
data = data[['protocol', 'src', 'src_port', 'dst', 'dst_port', 'packets', 'bytes', 'flow_duration', 'inter_time_avg',
             'response_rel_time', 'C_duration_avg', 'C_number_of_contacts', 'dst2src_packets', 'src2dst_packets_rate']]

In [10]:
categorical_columns = [c for c in data.columns if data[c].dtype.name == 'object'] #категоріальні фічі
numerical_columns   = [c for c in data.columns if data[c].dtype.name != 'object'] #некатегоріальні - числові

In [11]:
for c in categorical_columns:
    print (c, data[c].unique())

protocol ['TCP' 'UDP' 'ICMP']
src ['147.32.84.165' '195.113.232.73' '147.32.96.45' '90.177.113.3'
 '147.32.84.171' '203.106.50.35' '147.32.84.79' '94.63.149.150']
dst ['91.212.135.158' '147.32.80.9' '68.90.69.10' '88.255.110.57'
 '198.107.153.93' '94.245.120.86' '65.55.92.152' '147.32.84.165'
 '64.59.134.8' '128.107.234.207' '207.115.36.21' '205.232.66.196'
 '147.32.84.255' '195.50.106.142' '74.128.0.19' '202.108.252.141'
 '206.46.232.11' '65.54.188.126' '65.55.37.88' '204.232.236.215'
 '65.55.92.184' '24.123.245.98' '207.155.249.92' '173.166.213.146'
 '216.69.186.201' '74.54.45.130' '64.18.4.10' '204.232.236.234'
 '64.141.104.250' '207.111.237.10' '174.46.55.67' '170.224.49.13'
 '207.114.210.195' '204.232.236.138' '38.100.130.136' '207.126.147.12'
 '65.55.88.22' '216.35.196.26' '198.105.1.132' '202.81.31.148'
 '209.99.127.21' '202.131.203.240' '216.163.188.54' '64.191.223.38'
 '38.113.116.213' '65.54.188.72' '65.55.37.104' '216.99.131.17'
 '208.77.49.153' '69.5.251.52' '209.116.42.132

In [12]:
data.count(axis=0)

protocol                1492
src                     1492
src_port                1492
dst                     1492
dst_port                1492
packets                 1492
bytes                   1492
flow_duration           1492
inter_time_avg          1492
response_rel_time       1492
C_duration_avg          1492
C_number_of_contacts    1492
dst2src_packets         1492
src2dst_packets_rate    1492
dtype: int64

In [13]:
data_describe = data.describe(include=[object])

In [14]:
binary_columns = [c for c in categorical_columns if data_describe[c]['unique'] == 2]
nonbinary_columns = [c for c in categorical_columns if data_describe[c]['unique'] > 2]
print (binary_columns, nonbinary_columns)
# data_describe

[] ['protocol', 'src', 'dst']


In [15]:
for c in binary_columns: #найчастіші значення замінюємо на 0
    top = data_describe[c]['top']
    data.at[data[c] == top, c] = 0
    data.at[data[c] != 0, c] = 1

In [16]:
data_nonbinary = pd.get_dummies(data[nonbinary_columns], dummy_na=False)
data_nonbinary.astype('int', copy=False)
data_nonbinary.dtypes
# data_nonbinary

protocol_ICMP          uint8
protocol_TCP           uint8
protocol_UDP           uint8
src_147.32.84.165      uint8
src_147.32.84.171      uint8
src_147.32.84.79       uint8
src_147.32.96.45       uint8
src_195.113.232.73     uint8
src_203.106.50.35      uint8
src_90.177.113.3       uint8
src_94.63.149.150      uint8
dst_114.108.161.130    uint8
dst_115.119.181.169    uint8
dst_117.53.114.15      uint8
dst_118.221.122.4      uint8
dst_12.108.118.5       uint8
dst_12.147.30.2        uint8
dst_12.166.237.135     uint8
dst_12.200.228.4       uint8
dst_12.34.32.242       uint8
dst_125.5.96.59        uint8
dst_128.107.234.207    uint8
dst_128.125.253.79     uint8
dst_128.220.161.140    uint8
dst_128.96.20.41       uint8
dst_129.162.252.33     uint8
dst_129.41.169.246     uint8
dst_129.82.103.165     uint8
dst_130.225.242.145    uint8
dst_130.94.158.215     uint8
                       ...  
dst_82.194.66.207      uint8
dst_82.223.166.25      uint8
dst_83.100.190.162     uint8
dst_84.14.227.

In [17]:
data_numerical = data[numerical_columns]
data_numerical = (data_numerical - data_numerical.mean()) / data_numerical.std()
data_numerical.describe()

,src_port,dst_port,packets,bytes,flow_duration,inter_time_avg,response_rel_time,C_duration_avg,C_number_of_contacts,dst2src_packets,src2dst_packets_rate
count,1.492000e+03,1.492000e+03,1.492000e+03,1492.000000,1.492000e+03,1.492000e+03,1.492000e+03,1.492000e+03,1.492000e+03,1492.000000,1.492000e+03
mean,7.381644e-17,9.524702e-18,-2.381175e-17,0.000000,4.762351e-18,-1.190588e-18,4.762351e-18,9.524702e-18,-3.809881e-17,0.000000,2.619293e-17
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,1.000000e+00
min,-1.115206e+00,-3.921080e-01,-1.931374e-01,-0.113207,-7.672884e-02,-7.245589e-02,-7.808463e-02,-2.400369e-01,-3.277475e-01,-0.187979,-1.628662e-01
25%,-3.757158e-01,-3.809621e-01,-1.683387e-01,-0.109904,-7.672884e-02,-7.245589e-02,-7.808463e-02,-2.400369e-01,-3.277475e-01,-0.187979,-1.628662e-01
50%,-6.491562e-02,-3.809621e-01,-1.187413e-01,-0.103300,-6.322921e-02,-5.397706e-02,-7.808463e-02,-2.400369e-01,-3.277475e-01,-0.187979,-1.574110e-01
75%,3.298519e-01,-3.809621e-01,-1.187413e-01,-0.099625,-6.300206e-02,-5.367205e-02,-7.808463e-02,-2.400369e-01,-3.277475e-01,-0.187979,-1.574110e-01
max,2.156175e+01,1.911056e+01,3.546741e+01,35.519013,2.991158e+01,2.434440e+01,2.048895e+01,1.426615e+01,2.245172e+01,24.112521,1.074753e+01


In [21]:
datax = pd.concat((data_numerical, data[binary_columns], data_nonbinary), axis=1)
datax = pd.DataFrame(datax, dtype=float)
print (datax.shape)
print(list(datax.columns))

(1492, 586)
['src_port', 'dst_port', 'packets', 'bytes', 'flow_duration', 'inter_time_avg', 'response_rel_time', 'C_duration_avg', 'C_number_of_contacts', 'dst2src_packets', 'src2dst_packets_rate', 'protocol_ICMP', 'protocol_TCP', 'protocol_UDP', 'src_147.32.84.165', 'src_147.32.84.171', 'src_147.32.84.79', 'src_147.32.96.45', 'src_195.113.232.73', 'src_203.106.50.35', 'src_90.177.113.3', 'src_94.63.149.150', 'dst_114.108.161.130', 'dst_115.119.181.169', 'dst_117.53.114.15', 'dst_118.221.122.4', 'dst_12.108.118.5', 'dst_12.147.30.2', 'dst_12.166.237.135', 'dst_12.200.228.4', 'dst_12.34.32.242', 'dst_125.5.96.59', 'dst_128.107.234.207', 'dst_128.125.253.79', 'dst_128.220.161.140', 'dst_128.96.20.41', 'dst_129.162.252.33', 'dst_129.41.169.246', 'dst_129.82.103.165', 'dst_130.225.242.145', 'dst_130.94.158.215', 'dst_132.248.10.38', 'dst_133.163.13.13', 'dst_133.250.250.3', 'dst_133.40.6.10', 'dst_134.192.184.198', 'dst_134.83.127.98', 'dst_136.158.6.47', 'dst_136.234.10.21', 'dst_137.159.

In [19]:
datax.isnull().values.any()

False

In [248]:
# datax = data.copy()

In [249]:
# datax = datax[['protocol']]

In [130]:
# datax = datax[['protocol', 'src', 'src_port', 'dst', 'dst_port', 'packets', 'bytes', 'flow_duration', 'inter_time_avg',
#              'response_rel_time', 'C_duration_avg', 'C_number_of_contacts', 'dst2src_packets', 'src2dst_packets_rate']]

In [149]:
# datax['protocol'] = datax['protocol'].astype('category')
# datax['src'] = datax['src'].astype('category')
# datax['dst'] = datax['dst'].astype('category')

In [269]:
datax.dtypes

src_port                float64
dst_port                float64
packets                 float64
bytes                   float64
flow_duration           float64
inter_time_avg          float64
response_rel_time       float64
C_duration_avg          float64
C_number_of_contacts    float64
dst2src_packets         float64
src2dst_packets_rate    float64
protocol_ICMP           float64
protocol_TCP            float64
protocol_UDP            float64
src_147.32.84.165       float64
src_147.32.84.171       float64
src_147.32.84.79        float64
src_147.32.96.45        float64
src_195.113.232.73      float64
src_203.106.50.35       float64
src_90.177.113.3        float64
src_94.63.149.150       float64
dst_114.108.161.130     float64
dst_115.119.181.169     float64
dst_117.53.114.15       float64
dst_118.221.122.4       float64
dst_12.108.118.5        float64
dst_12.147.30.2         float64
dst_12.166.237.135      float64
dst_12.200.228.4        float64
                         ...   
dst_82.1

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(datax, res, test_size = 0.3, random_state = 11)

Nx_train, _ = X_train.shape 
Nx_test,  _ = X_test.shape
Ny_train = y_train.shape 
Ny_test = y_test.shape 
print (Nx_train, Nx_test)
print (Ny_train, Ny_test)

1044 448
(1044,) (448,)


In [23]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

# y_train_predict = knn.predict(X_train)
# y_test_predict = knn.predict(X_test)

# err_train = np.mean(y_train != y_train_predict)
# err_test  = np.mean(y_test  != y_test_predict)
# print (err_train, err_test)
print(classification_report(y_train, knn.predict(X_train)))
print(classification_report(y_test, knn.predict(X_test)))

             precision    recall  f1-score   support

          0       0.97      0.91      0.94        67
          1       0.99      1.00      1.00       977

avg / total       0.99      0.99      0.99      1044

             precision    recall  f1-score   support

          0       0.97      1.00      0.98        28
          1       1.00      1.00      1.00       420

avg / total       1.00      1.00      1.00       448



In [24]:
from sklearn.svm import SVC #SVM!

svc = SVC()
svc.fit(X_train, y_train)

# err_train = np.mean(y_train != svc.predict(X_train))
# err_test  = np.mean(y_test  != svc.predict(X_test))
# print (err_train, err_test)

print(classification_report(y_train, svc.predict(X_train)))
print(classification_report(y_test, svc.predict(X_test))) #

             precision    recall  f1-score   support

          0       1.00      0.42      0.59        67
          1       0.96      1.00      0.98       977

avg / total       0.96      0.96      0.96      1044

             precision    recall  f1-score   support

          0       1.00      0.46      0.63        28
          1       0.97      1.00      0.98       420

avg / total       0.97      0.97      0.96       448



In [25]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

print(classification_report(y_train, clf.predict(X_train)))
print(classification_report(y_test, clf.predict(X_test))) 

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        67
          1       1.00      1.00      1.00       977

avg / total       1.00      1.00      1.00      1044

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        28
          1       1.00      1.00      1.00       420

avg / total       1.00      1.00      1.00       448



In [26]:
from sklearn import ensemble

rf = ensemble.RandomForestClassifier(n_estimators=82, random_state=10)
rf.fit(X_train, y_train)

err_train = np.mean(y_train != rf.predict(X_train))
err_test  = np.mean(y_test  != rf.predict(X_test))
print (err_train, err_test)

print(classification_report(y_train, rf.predict(X_train)))
print(classification_report(y_test, rf.predict(X_test))) #

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        67
          1       1.00      1.00      1.00       977

avg / total       1.00      1.00      1.00      1044

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        28
          1       1.00      1.00      1.00       420

avg / total       1.00      1.00      1.00       448



In [27]:
gbt = ensemble.GradientBoostingClassifier(n_estimators=105, random_state=None)
gbt.fit(X_train, y_train)

err_train = np.mean(y_train != gbt.predict(X_train))
err_test = np.mean(y_test != gbt.predict(X_test))
print (err_train, err_test)

print(classification_report(y_train, gbt.predict(X_train)))
print(classification_report(y_test, gbt.predict(X_test))) #

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        67
          1       1.00      1.00      1.00       977

avg / total       1.00      1.00      1.00      1044

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        28
          1       1.00      1.00      1.00       420

avg / total       1.00      1.00      1.00       448



In [28]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, 
                    hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X_train, y_train)

err_train = np.mean(y_train != clf.predict(X_train))
err_test = np.mean(y_test != clf.predict(X_test))
print (err_train, err_test)

print(classification_report(y_train, clf.predict(X_train)))
print(classification_report(y_test, clf.predict(X_test))) #

             precision    recall  f1-score   support

          0       0.96      1.00      0.98        67
          1       1.00      1.00      1.00       977

avg / total       1.00      1.00      1.00      1044

             precision    recall  f1-score   support

          0       0.93      1.00      0.97        28
          1       1.00      1.00      1.00       420

avg / total       1.00      1.00      1.00       448



In [29]:
# Stochastic Gradient Descent

from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss="hinge", penalty="l1")
#clf = SGDClassifier(loss="log")
clf.fit(X_train, y_train)

err_train = np.mean(y_train != clf.predict(X_train))
err_test = np.mean(y_test != clf.predict(X_test))
print (err_train, err_test)

print(classification_report(y_train, clf.predict(X_train)))
print(classification_report(y_test, clf.predict(X_test))) #

             precision    recall  f1-score   support

          0       1.00      0.96      0.98        67
          1       1.00      1.00      1.00       977

avg / total       1.00      1.00      1.00      1044

             precision    recall  f1-score   support

          0       0.96      0.96      0.96        28
          1       1.00      1.00      1.00       420

avg / total       1.00      1.00      1.00       448



In [30]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(X_train, y_train)

print(classification_report(y_train, gnb.predict(X_train)))
print(classification_report(y_test, gnb.predict(X_test))) #

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        67
          1       1.00      1.00      1.00       977

avg / total       1.00      1.00      1.00      1044

             precision    recall  f1-score   support

          0       0.16      1.00      0.27        28
          1       1.00      0.64      0.78       420

avg / total       0.95      0.66      0.75       448

